In [1]:
import keras
import anago

Using TensorFlow backend.


In [2]:
# 챗봇 4단계 모델 불러오기
weights = './ner_model_test/mecab_model_weights'
params = './ner_model_test/mecab_model_params'
preprocessor = './ner_model_test/mecab_model_preprocessor'
model = anago.Sequence.load(weights, params, preprocessor)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [3]:
# 형태소 분석기 불러오기
from konlpy.tag import Okt
#okt = Okt()
from konlpy.tag import Mecab
mecab = Mecab()

In [4]:
# 챗봇 3단계 모델 불러오기
import pickle
with open('./ner_model_test/classifier_tfidf_3.pickle', "rb") as f:
    classifier_tfidf = pickle.load(f)
with open("./ner_model_test/mlb_3.pickle", "rb") as f:
    mlb = pickle.load(f)
with open("./ner_model_test/tfidf_vectorizer_3.pickle", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

/home/aperio1987a/anaconda3/envs/data-anal/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/aperio1987a/anaconda3/envs/data-anal/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.3 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/aperio1987a/anaconda3/envs/data-anal/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 0.21.3 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/aperio1987a/anaconda3/envs/data-anal/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator

In [5]:
import re

In [6]:
# 챗봇 3단계 모델 실행에 필요한 함수 정의
QUESTION_RE = re.compile('[^ ㄱ-ㅣ가-힣a-zA-Z]+')

STOPWORDS = set(['은','는','이','가','하','아','것','들','의','그','수','한','나','같','그렇'
                ,'문제','그리고','크','중','나오','지금','생각하','집','어떤','명','생각','이런'
                ,'인','지','을','를','에','스러운','스러워','주','할','만','게','도','져','된','로','고','던','로운','면서'
                ,'사실','이렇','점','싶','말','좀','식당','가게','집','음식점'
                ,'는지','나요','해요','해','는가요','삼','게요','예','는가','습니까','죠','려고요','는지요','서요','였어요','겠'
                ,'인가요','요' '라는','데','해서','세요','어요','을까요','건가요','겠죠','실래요','네요','으세요','지요','인데요'
                ,'드려요','려구요','합니다'])

def text_prepare(text):
    
    # 한글과 스페이스바를 제외한 다른 문자들은 모두 지운다.
    text = QUESTION_RE.sub('', text) 
    
    # mecab 토크나이저
    mecab = Mecab()
    
    # mecab으로 text를 형태소 단위로 나누고 불용어를 지운다.
    text = ' '.join(token for token in mecab.morphs(text) if token not in STOPWORDS)

    return text

#### 테스트용 질문 제작 (분석 잘된거/안된거, 각각 3개씩, 총 102개(=51*2))

***추천 요청 질문 말뭉치*** 
- `음식메뉴` : train 메뉴리스트에 있는 것/없는 것 둘 다 사용해볼 것
- `형용사`,`장소`,`어미` : 실제로 사용자들이 챗봇에게 물어볼만한 말투

- 1) 음식메뉴 + 추천요청어미
- 2) 음식메뉴 + 형용사 + 추천요청어미
- 3) 음식메뉴 + 형용사 + 장소 + 추천요청어미
- 4) 음식메뉴 + 장소 + 형용사 + 추천요청어미
- 5) 형용사 + 추천요청어미
- 6) 형용사 + 음식메뉴 + 추천요청어미
- 7) 형용사 + 형용사 + 추천요청어미
- 8) 형용사 + 음식메뉴 + 장소 + 추천요청어미
- 9) 장소 + 추천요청어미
- 10) 장소 + 음식메뉴 + 추천요청어미
- 11) 장소 + 형용사 + 추천요청어미
- 12) 장소 + 형용사 + 음식메뉴 + 추천요청어미

***문의(장소,리뷰) 질문 말뭉치***
- `가게명` : 골목식당 가게리스트 사용
- `형용사`,`장소`,`어미` : 실제로 사용자들이 챗봇에게 물어볼만한 말투

- 1) 가게명 + 장소문의어미
- 2) 가게명 + 리뷰문의어미
- 3) 가게명 + 형용사 + 리뷰문의어미
- 4) 장소 + 가게명 + 리뷰문의어미
- 5) 장소 + 가게명 + 형용사 + 리뷰문의어미

1. 먼저 아래 셀에서 `text`를 바꿔가며 여러 질문을 테스트한다.
2. 형용사/음식메뉴/장소/가게명의 분석이 잘 된 것은 `X_good`, 뭔가 하나라도 실패한 것은 `X_bad` list에 저장한다.
    - 형용사 : `ADJ`, 음식메뉴 :  `FOD`, 장소 : `LOC`, 가게명 : `ORG`
    
    - `X_good`,`X_bad` list 안의 질문의 개수는 각각 51개씩, (더 많아도 상관 없음)
3. `y_good`,`y_bad` list 안에 각각의 tag를 단다. 
    - (1) : recommendation/questionLOC/questionREV
    - (2) : taste/sanitation/price/atmosphere/prefer
    - (3) : korean/chinese/western/japanese/chicken/asian/etc
    - 태그를 달 때 (1)은 반드시 포함해야 한다. (2),(3)은 포함할 수도 있고 포함하지 않을 수도 있다.
    - (1),(2),(3) 각 태그 종류에서 1개 태그만 선택해야 한다. (korean,chinese를 동시에 달 수 없음)
4. `X_good`, `X_bad` 각각에서 챗봇 4단계, 챗봇 3단계 모델을 실행하고 결과를 저장한다.
5. `okt`/`mecab` 비교

In [30]:
# 1. text를 바꿔가며 테스트
text ='거제에 풍년비빔밥 재료가 신선한지 말해주세요'
text_prepared = ' '.join(word for word in mecab.morphs(text))
model.analyze(text_prepared)

{'words': ['거제', '에', '풍년', '비빔밥', '재료', '가', '신선', '한지', '말', '해', '주', '세요'],
 'entities': [{'text': '거제',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '풍년 비빔밥',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '재료 가 신선 한지',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 8}]}

In [67]:
# 2. 1번에서 형용사/음식메뉴/장소/가게명의 분석이 잘 되는 질문은 X_good, 잘 안되면 X_bad에 담는다.
X_good = [
    '연돈 위치 알려줘'
    ,'타코얔 어디에 있어?'
    ,'홍백떡볶이 주소 말해줘'
    ,'온센텐동 맛 어때?'
    ,'그린몬스터 후기 알려줘'
    ,'필동멸치국수 괜찮아?'
    ,'코너스테이크 분위기 어땠어?'
    ,'부라더쭈꾸미 많이 매워?'
    ,'생선보감 생선 비린 맛 어때?'
    ,'용산에 코스모스 어때?' 
    ,'부천에 있는 피자앤펍 후기 알려줘'
    ,'원주에서 부리또만나 리뷰 보여줘'
    ,'원주에 부리또만나 리뷰 보여줘'
    ,'전라도 여수 꼬치는맛 많이 매운지 말해줘'
    ,'서울에 있는 준까스 파삭파삭한지 알려줘'
     '똠양꿍 추천해줘'
    ,'국밥 알려줘'
    ,'크림파스타 알려줘' 
    ,'샌드위치가게 추천'
    ,'파스타 종류 뭐뭐 있어?'
    ,'해물탕 재료가 좋은 곳 추천'
    ,'국밥집 국물이 시원한 곳 알려줘'
    ,'초밥집 재료가 신선한 곳 어디야?'
    ,'치킨 포장할 수 있는 곳 알려줘'
    ,'순살치킨 배달해주는 곳 알려줘'
    ,'김치찌개 국물이 시원한 곳 경기도 안양 추천'
    ,'닭갈비 제일 유명한 곳 춘천에 있는 곳으로 찾아줘'
    ,'마카롱 맛있는 곳 부산에 있는 곳으로 추천해줘'
    ,'막국수 춘천에 안 매운 곳 말해줘'
    ,'피자 인천에 있고 주차할 수 있는 곳으로 알려줘'   
    
#     ,'한정식 분위기 좋고 깔끔한 곳 대전 기준'
#     ,'맛있는 거 추천해줘'
#     ,'조용한 곳 추천해줘'
#     ,'소개팅을 하기 좋은 맛집 알려줘'
#     ,'조미료 안 쓰는 죽집 찾아줘'    
#     ,'가격이 저렴한 반찬가게 찾아줘'
#     ,'맛있는 제육덮밥 맛집 알려줘'
#     ,'싱싱한 장어집 알려줘'
#     ,'싱싱한 회 파는 곳 안내해줘'
#     ,'양 푸짐한 찜닭집 추천ㄱㄱ'
#     ,'정갈한 한식집 추천해줘'
#     ,'단체석 있는 고기집 추천'
#     ,'비린맛 안 나는 생선구이집 알려줘'
#     ,'누린내없는 족발집 알려줘'
#     ,'분위기 좋은 파스타집 알려줘'
#     ,'육즙 많이 흐르는 고기집 추천해줘'
#     ,'줄 서서 기다리는 마라탕 집 있을까요'
#     ,'깔끔한 인도 레스토랑 알려줘'
#     ,'분위기 좋고 조명이 은은한 포토존 맛집 추천'
#     ,'향이 좋고 맛이 특이한 곳 알려줘'
#     ,'공기 좋고 자리 넓은 데 추천'
#     ,'몸보신하기 좋은 백숙집 인천에서 추천'
#     ,'고기를 많이 주는 국밥집 부산에 있는 곳으로 찾아줘' 
#     ,'가성비 좋은 짬뽕집 부산에서 찾아줘'
#     ,'인천에는 뭐가 맛있나'
#     ,'울산에서 뭐먹지'
#     ,'안동 맛집'
#     ,'영덕에 대게 맛집'
#     ,'회식 장소로 인기있는 족발집 알려줘'
#     ,'강릉 맛있는 횟집 추천'
#     ,'여수에서 주차무료인 밥집 추천'
#     ,'종각에서 고기가 가장 두툼한 돈까스집은?'
#     ,'종로에 분위기 좋은 순대국밥집 있나요?'
#     ,'전주에 맛있는 비빔밥집 있나요?'
#     ,'조용한 이탈리안 레스토랑 알려줘'
#     ,'밥 많이 주는 한식집 알려주셈'
#     ,'마르게리따 피자가 맛있는 이탈리안레스토랑 추천'
#     ,'마라탕 맵지 않게 만들어주는 중국 음식점 찾아줘'
]

X_bad = [
    '소개팅 하기 좋은 맛집 알려줘'
    ,'맛있는 김밥 찾아줘'
    ,'매운 갈비찜 찾아줘'
    ,'제일 매운 갈비찜이 뭐예요?'
    ,'주차 되고 넓은 곳 찾아줘'
    ,'정갈하고 상견례하기 좋은 곳 있을까?' 
    ,'맛 좋고 맛이 특이한 곳 알려줘'
    ,'인테리어 멋지고 사진 잘 나오는 맛집' 
    ,'공기 좋고 넓은 데 추천' 
    ,'몸보신하기 좋은 백숙집 인천에서 추천'
    ,'고기 많이 들어간 국밥집 부산에 있는 걸로 찾아줘' 
    ,'원조 인도 카레 맛집 이태원에서 추천' 
    ,'향긋한 차 마실 수 있는 곳 용인으로 알려줘' 
    ,'광주 카페 추천'
    ,'서울은 뭐가 맛있어?'
    ,'대전에서 뭘 먹어야 할까?'
    ,'제주도 흙돼지구이 추천' 
    ,'전라도 보쌈 맛집 추천' 
    ,'제주도 흙돼지구이 추천'
    ,'광주에서 부모님과 가기 좋은 레스토랑 추천' 
    ,'야경 예쁘고 사진 잘 나오는 카페 알려줘' 
    ,'정갈하면서 깔끔한 한정식 집 알려줘'
    ,'부라더쭈꾸미 많이 매워?'
    ,'용산 코스모스 어때?' 
    ,'부천 피자앤펍 후기 알려줘'
    ,'원주에서 부리또만나 리뷰 보여줘'
    ,'서울 준까스 파삭파삭한지 알려줘'
    ,'서대문 홍탁집 대기줄 길어?'
    ,'강동구 더짬뽕 양 많이 주나요?'
    ,'거제에 풍년비빔밥 재료가 신선한지 말해주세요'
]

In [68]:
# 3. X_good과 X_bad 각각에 태그 y_good, y_bad를 단다.
y_good = [
     ['questionLOC']
    ,['questionLOC']
    ,['questionLOC']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['recommendation', 'prefer']
    ,['recommendation']
    ,['recommendation']
    ,['recommendation']
    ,['recommendation']
    ,['recommendation','taste']
    ,['recommendation','taste']
    ,['recommendation','taste']
    ,['recommendation']
    ,['recommendation']
    ,['recommendation','taste','korean']
    ,['recommendation','prefer','korean']
    ,['recommendation','taste','etc']
    ,['recommendation','taste','korean']
#     ,['recommendation','western']
#     ,['recommendation','sanitation','korean']
#     ,['recommendation','taste']
#     ,['recommendation','atmosphere']
#     ,['recommendation','atmosphere']
#     ,['recommendation','taste','korean']  
#     ,['recommendation','price','etc'] 
#     ,['recommendation','taste','korean']  
#     ,['recommendation','taste','korean'] 
#     ,['recommendation','taste','japanese']  
#     ,['recommendation','taste','korean'] 
#     ,['recommendation','atmosphere','korean']  
#     ,['recommendation','korean']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','atmosphere','western']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','prefer','chinese']  
#     ,['recommendation','sanitation','asian']  
#     ,['recommendation','atmosphere']  
#     ,['recommendation','taste']  
#     ,['recommendation','atmosphere']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','price','chinese']  
#     ,['recommendation']  
#     ,['recommendation']
#     ,['recommendation']
#     ,['recommendation','korean']  
#     ,['recommendation','prefer','korean']  
#     ,['recommendation','taste','japanese']  
#     ,['recommendation','korean']  
#     ,['recommendation','taste','japanses']  
#     ,['recommendation','atmosphere','korean']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','atmosphere','western']  
#     ,['recommendation','taste','korean']  
#     ,['recommendation','taste','wsetern']  
#     ,['recommendation','taste','chinese']   
]
y_bad = [
     ['recommendation','atmospher']
    ,['recommendation','taste','korean']
    ,['recommendation','taste','korean']
    ,['recommendation','taste','korean']
    ,['recommendation','atmosphere']
    ,['recommendation','atmosphere']
    ,['recommendation','taste']
    ,['recommendation','atmosphere']
    ,['recommendation','atmosphere']
    ,['recommendation','taste']
    ,['recommendation','taste','korean']
    ,['recommendation','taste','asian']
    ,['recommendation','taste','etc']
    ,['recommendation','etc']
    ,['recommendation','taste']
    ,['recommendation']
    ,['recommendation']
    ,['recommendation','prefer']
    ,['recommendation','prefer']
    ,['recommendation','atmosphere','western']
    ,['recommendation','atmosphere','etc']
    ,['recommendation','atmosphere','korean']
    ,['recommendation','taste','korean']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
    ,['questionREV']
]

In [69]:
# 3. 예시
X_chatbot = [
     '맛있는 파스타집 추천해주세요'
    , '종로에 분위기 좋은 순대국밥집 있나요?'
    , '청결한 한식집 있어요?'
    , '가성비 좋은 짬뽕 집 알려주세요'
    , '줄 서서 기다리는 마라탕 집 있을까요'
    , '보쌈집 위치 알려주십시오'
    , '언니네횟집 주소좀요'
    , '언니네횟집 후기 좀 알려주세요'
    , '언니네횟집 사람들 반응점'
]
y_chatbot = [
    ['recommendation','taste','western']
    ,['recommendation','atmosphere','korean']
    ,['recommendation','sanitation','korean']
    ,['recommendation','price','chinese']
    ,['recommendation','prefer','chinese']
    ,['questionLOC']
    ,['questionLOC']
    ,['questionREV']
    ,['questionREV']
]

In [70]:
# 챗봇 3단계 모델 실행결과

# SENTENCE 데이터 전처리
X_good_prepared = [text_prepare(x) for x in X_good]
X_bad_prepared = [text_prepare(x) for x in X_bad]

In [71]:
# tf-idf 벡터화
X_good_tfidf = tfidf_vectorizer.transform(X_good)
X_bad_tfidf = tfidf_vectorizer.transform(X_bad)

In [72]:
# tag 데이터 전처리
y_good = mlb.transform(y_good)
y_bad = mlb.transform(y_bad)

In [73]:
# tag 예측
y_good_predicted_labels_tfidf = classifier_tfidf.predict(X_good_tfidf)
y_good_predicted_scores_tfidf = classifier_tfidf.decision_function(X_good_tfidf)
y_bad_predicted_labels_tfidf = classifier_tfidf.predict(X_bad_tfidf)
y_bad_predicted_scores_tfidf = classifier_tfidf.decision_function(X_bad_tfidf)

In [74]:
# 예측된 tag를 텍스트로 변환
y_good_pred_inversed = mlb.inverse_transform(y_good_predicted_labels_tfidf)
y_good_inversed = mlb.inverse_transform(y_good)
y_bad_pred_inversed = mlb.inverse_transform(y_bad_predicted_labels_tfidf)
y_bad_inversed = mlb.inverse_transform(y_bad)

In [75]:
for i in range(0,len(y_good)):
    print('SENTENCE:\t{}\n정답:\t{}\n예측:\t{}\n\n'.format(
        X_good[i],
        y_good_inversed[i],
        y_good_pred_inversed[i]
    ))
for i in range(0,len(y_bad)):
    print('SENTENCE:\t{}\n정답:\t{}\n예측:\t{}\n\n'.format(
        X_bad[i],
        y_bad_inversed[i],
        y_bad_pred_inversed[i]
    ))

SENTENCE:	연돈 위치 알려줘
정답:	('questionLOC',)
예측:	('questionLOC',)


SENTENCE:	타코얔 어디에 있어?
정답:	('questionLOC',)
예측:	('questionLOC',)


SENTENCE:	홍백떡볶이 주소 말해줘
정답:	('questionLOC',)
예측:	('questionLOC',)


SENTENCE:	온센텐동 맛 어때?
정답:	('questionREV',)
예측:	('recommendation', 'taste')


SENTENCE:	그린몬스터 후기 알려줘
정답:	('questionREV',)
예측:	('questionREV',)


SENTENCE:	필동멸치국수 괜찮아?
정답:	('questionREV',)
예측:	('questionLOC',)


SENTENCE:	코너스테이크 분위기 어땠어?
정답:	('questionREV',)
예측:	('atmosphere', 'recommendation')


SENTENCE:	부라더쭈꾸미 많이 매워?
정답:	('questionREV',)
예측:	('recommendation', 'taste')


SENTENCE:	생선보감 생선 비린 맛 어때?
정답:	('questionREV',)
예측:	('korean', 'recommendation', 'taste')


SENTENCE:	용산에 코스모스 어때?
정답:	('questionREV',)
예측:	('questionLOC',)


SENTENCE:	부천에 있는 피자앤펍 후기 알려줘
정답:	('questionREV',)
예측:	('questionREV',)


SENTENCE:	원주에서 부리또만나 리뷰 보여줘
정답:	('questionREV',)
예측:	('questionREV',)


SENTENCE:	원주에 부리또만나 리뷰 보여줘
정답:	('questionREV',)
예측:	('questionREV',)


SENTENCE:	전라도 여수 꼬치는맛 많이 매운지 말해줘
정답:	('questionREV',)


In [76]:
# 챗봇 4단계 모델 실행결과 (X_good)
for text in X_good:
    text_prepared = ' '.join(word for word in mecab.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

{'words': ['연', '돈', '위치', '알려줘'],
 'entities': [{'text': '연 돈',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2}]}

{'words': ['타코', '얔', '어디', '에', '있', '어', '?'],
 'entities': [{'text': '타코 얔',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2}]}

{'words': ['홍백', '떡볶이', '주소', '말', '해', '줘'],
 'entities': [{'text': '홍백 떡볶이',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2}]}

{'words': ['온', '센', '텐동', '맛', '어때', '?'],
 'entities': [{'text': '온 센 텐동 맛 어때 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 6}]}

{'words': ['그린', '몬스터', '후기', '알려줘'],
 'entities': [{'text': '그린 몬스터',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2}]}

...

{'words': ['김치찌개', '국물', '이', '시원', '한', '곳', '경기도', '안양', '추천'],
 'entities': [{'text': '김치찌개',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '국물 이 시원 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 5},
  {'text': '경기도 안양',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 6,
   'endOffset': 8}]}

{'words': ['닭갈비',
  '제일',
  '유명',
  '한',
  '곳',
  '춘천',
  '에',
  '있',
  '는',
  '곳',
  '으로',
  '찾',
  '아',
  '줘'],
 'entities': [{'text': '닭갈비',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '제일 유명 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 4},
  {'text': '춘천',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 6}]}

{'words': ['마카롱',
  '맛있',
  '는',
  '곳',
  '부산',
  '에',
  '있',
  '는',
  '곳',
  '으로',
  '추천',
  '해',
  '줘'],
 'entities': [{'text': '마카롱',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 3},
  {'text': '부산',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 5}]}

{'words': ['막국수', '춘천', '에', '안', '매운', '곳', '말', '해', '줘'],
 'entities': [{'text': '막국수',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '춘천',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 2},
  {'text': '안 매운',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 3,
   'endOffset': 5}]}

{'words': ['피자',
  '인천',
  '에',
  '있',
  '고',
  '주차',
  '할',
  '수',
  '있',
  '는',
  '곳',
  '으로',
  '알려줘'],
 'entities': [{'text': '피자',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '인천',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 2},
  {'text': '주차 할 수 있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 10}]}

In [77]:
# 챗봇 4단계 모델 실행결과 (X_bad)
for text in X_bad:
    text_prepared = ' '.join(word for word in mecab.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

{'words': ['소개팅', '하', '기', '좋', '은', '맛집', '알려줘'],
 'entities': [{'text': '소개팅',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '하 기 좋 은',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 5}]}

{'words': ['맛있', '는', '김밥', '찾', '아', '줘'],
 'entities': [{'text': '맛있 는 김밥',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 3}]}

{'words': ['매운', '갈비찜', '찾', '아', '줘'],
 'entities': [{'text': '매운 갈비찜',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2}]}

{'words': ['제일', '매운', '갈비찜', '이', '뭐', '예요', '?'],
 'entities': [{'text': '제일 매운 갈비찜 이 뭐 예요 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 7}]}

{'words': ['주차', '되', '고', '넓', '은', '곳', '찾', '아', '줘'],
 'entities': [{'text': '주차 되 고',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 3},
  {'text': '넓 은',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 3,
   'endOffset': 5}]}

...

{'words': ['원주', '에서', '부리', '또', '만나', '리뷰', '보여줘'],
 'entities': [{'text': '원주 에서 부리 또 만나',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 5}]}

{'words': ['서울', '준', '까스', '파삭파삭', '한지', '알려줘'],
 'entities': [{'text': '서울 준 까스',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 3},
  {'text': '파삭파삭 한지 알려줘',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 3,
   'endOffset': 6}]}

{'words': ['서대문', '홍', '탁', '집', '대기', '줄', '길', '어', '?'],
 'entities': [{'text': '서대문 홍 탁 집 대기 줄 길 어 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 9}]}

{'words': ['강동구', '더', '짬뽕', '양', '많이', '주', '나요', '?'],
 'entities': [{'text': '강동구 더 짬뽕 양 많이 주 나요 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 8}]}

{'words': ['거제', '에', '풍년', '비빔밥', '재료', '가', '신선', '한지', '말', '해', '주', '세요'],
 'entities': [{'text': '거제',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '풍년 비빔밥',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '재료 가 신선 한지',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 8}]}

In [78]:
q_list = [
    '맛있는 제육덮밥 집 알려줘'
    ,'파스쿠찌 위치 알려주세요'
    ,'담소 순대국밥집 청결한가요?'
    ,'담소 순대국밥집 맛있어요?'
    ,'담소 순대국밥집 맛있는 곳인가요?'
    ,'숯불구이 맛있는곳 알려주실래요?'
    ,'온센텐동 사람들 리뷰 좀 알려주세요'
    ,'온센텐동 위치가 어떻게 되나요'
    ,'파스쿠찌 맛있어요?'
    ,'가성비 대박인 육개장 집 알려주세요'
    ,'맛있는 육개장집 알려주세요'
    ,'육개장 맛있는 곳 알려주세요'
    ,'서울에 맛있는 육개장 가게 알려주세요'
    ,'제주도에 유명한 흑돼지 식당 알려주세요'
    ,'제주도에 맛있는 흑돼지 식당 알려주세요'
    ,'제주도에 흑돼지 식당 맛있는 곳 알려주세요'
    ,'제주도에 흑돼지 식당 맛집 알려주세요'
    ,'제주도에 흑돼지 식당 맛 개쩌는 곳 알려주세요'
    ,'맛있고 깔끔한 한식집 아세요'
    ,'맛있는 깔끔한 한식집 아세요'
]

In [79]:
# mecab 실행결과
for text in q_list:
    text_prepared = ' '.join(word for word in mecab.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

{'words': ['맛있', '는', '제육', '덮밥', '집', '알려줘'],
 'entities': [{'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2},
  {'text': '제육 덮밥',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '집 알려줘',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

{'words': ['파스', '쿠', '찌', '위치', '알려', '주', '세요'],
 'entities': [{'text': '파스 쿠 찌',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 3}]}

{'words': ['담소', '순대', '국밥', '집', '청결', '한가요', '?'],
 'entities': [{'text': '담소 순대 국밥 집',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '청결 한가요',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

{'words': ['담소', '순대', '국밥', '집', '맛있', '어요', '?'],
 'entities': [{'text': '담소 순대 국밥 집 맛있 어요 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 7}]}

{'words': ['담소', '순대', '국밥', '집', '맛있', '는', '곳', '인가요', '?'],
 'entities': [{'text': '담소 순대 국밥 집',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

...

{'words': ['제주도', '에', '흑', '돼지', '식당', '맛있', '는', '곳', '알려', '주', '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5},
  {'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 7}]}

{'words': ['제주도', '에', '흑', '돼지', '식당', '맛집', '알려', '주', '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5}]}

{'words': ['제주도',
  '에',
  '흑',
  '돼지',
  '식당',
  '맛',
  '개',
  '쩌',
  '는',
  '곳',
  '알려',
  '주',
  '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5},
  {'text': '맛 개 쩌 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 9}]}

{'words': ['맛있', '고', '깔끔', '한', '한식집', '아세요'],
 'entities': [{'text': '맛있 고 깔끔 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '한식집',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 5}]}

{'words': ['맛있', '는', '깔끔', '한', '한식집', '아세요'],
 'entities': [{'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2},
  {'text': '깔끔 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '한식집',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 5}]}

In [80]:
# mecab 실행결과
for text in q_list:
    text_prepared = ' '.join(word for word in mecab.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

{'words': ['맛있', '는', '제육', '덮밥', '집', '알려줘'],
 'entities': [{'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2},
  {'text': '제육 덮밥',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '집 알려줘',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

{'words': ['파스', '쿠', '찌', '위치', '알려', '주', '세요'],
 'entities': [{'text': '파스 쿠 찌',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 3}]}

{'words': ['담소', '순대', '국밥', '집', '청결', '한가요', '?'],
 'entities': [{'text': '담소 순대 국밥 집',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '청결 한가요',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

{'words': ['담소', '순대', '국밥', '집', '맛있', '어요', '?'],
 'entities': [{'text': '담소 순대 국밥 집 맛있 어요 ?',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 7}]}

{'words': ['담소', '순대', '국밥', '집', '맛있', '는', '곳', '인가요', '?'],
 'entities': [{'text': '담소 순대 국밥 집',
   'type': 'ORG',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 6}]}

...

{'words': ['제주도', '에', '흑', '돼지', '식당', '맛있', '는', '곳', '알려', '주', '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5},
  {'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 7}]}

{'words': ['제주도', '에', '흑', '돼지', '식당', '맛집', '알려', '주', '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5}]}

{'words': ['제주도',
  '에',
  '흑',
  '돼지',
  '식당',
  '맛',
  '개',
  '쩌',
  '는',
  '곳',
  '알려',
  '주',
  '세요'],
 'entities': [{'text': '제주도',
   'type': 'LOC',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '흑 돼지 식당',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 5},
  {'text': '맛 개 쩌 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 5,
   'endOffset': 9}]}

{'words': ['맛있', '고', '깔끔', '한', '한식집', '아세요'],
 'entities': [{'text': '맛있 고 깔끔 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 4},
  {'text': '한식집',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 5}]}

{'words': ['맛있', '는', '깔끔', '한', '한식집', '아세요'],
 'entities': [{'text': '맛있 는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 2},
  {'text': '깔끔 한',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 2,
   'endOffset': 4},
  {'text': '한식집',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 4,
   'endOffset': 5}]}